In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import validators

In [12]:
def get_links_from_url(uset):
    try:
        html = urlopen(uset['url'], timeout=10)
        html_data = html.read()
    except:
        return None
    bsObj = BeautifulSoup(html_data, 'html.parser')
    links_url = set()
    for link in bsObj.find_all('a', href=re.compile('^http')):
        if 'href' in link.attrs:
            if not validators.url(link.attrs['href']):
                continue
            links_url.add(link.attrs['href'])
    if len(links_url) == 0:
        return None
    else:
        return links_url

In [29]:
def parse_content(url):
    try:
        html = urlopen(url)
        html_data = html.read()
        bs = BeautifulSoup(html_data, 'html5lib')
        
        fb_metas = bs.find_all('meta', attrs={'property':True})
        cats = bs.select('.basicListType ul li a')
        author_trl = bs.select('span.gd_auth a')
        pub_corp = bs.select('span.gd_pub a')
        pub_date = bs.select('span.gd_date')
        pub_ori_title = bs.select('span.gd_origin a')
        book_cont = {}
        prd_sdata =  bs.select('#contents .basicListType td.cell_2col')
        cm = bs.select('div.communtyHide')
    except:
        return False
    # Facebook og_metas
    og_metas = {}
    for m in fb_metas:
        og_metas[str(m['property'])] = m['content']

    # book_cateogory
    bcats = []
    for c in cats:
        bcats.append(c.text)

    bauthors = []
    for p in author_trl:
        bauthors.append(p.text)

    bcorp =  None if pub_corp == None or len(pub_corp)==0 else pub_corp[0].text
    bdate = None if pub_date == None or len(pub_date)==0 else pub_date[0].text

    # print(og_metas, bcats, bauthors, bcorp,bdate)
    bsdata = {}
    for idx,tag in enumerate(prd_sdata):
        tag_text = tag.text
        if idx == 1:
            bsdata['bibli'] = re.sub(r'(\n|\s\s)','', tag_text.strip())
        if idx == 2 and tag_text.isdigit() and validators.length(tag_text,min=13,max=13):
            bsdata['isbn13'] = tag_text
        else :
            bsdata['isbn13'] = None
        if idx == 3  and tag_text.isdigit() and validators.length(tag_text,min=10,max=10):
            bsdata['isbn10'] = tag_text
        else :
            bsdata['isbn10'] = None
    bdata = {}
    for k in og_metas:
        bdata[k]=og_metas[k]

    for k in bsdata:
        bdata[k]=bsdata[k]

    bdata['cats'] = ",".join(bcats)
    bdata['authors'] = ",".join(bauthors)
    bdata['corp'] = bcorp
    bdata['pub_date'] = bdate


    # 책 간략 설명 텍스트 파싱
    if bs.find('a',attrs={'name':'contentsIntro'}) != None:
        t_cnt = 0
        for tag in bs.find('a',attrs={'name':'contentsIntro'}).next_elements:
            t_cnt +=1
            if(t_cnt==2):
                book_intro = re.sub(r'(\n|\s\s)','',tag.text.strip())
                break
    else :
        book_intro = None

    if bs.select('#contents_author') != None and len(bs.select('#contents_author')) != 0:
        author_intro = re.sub(r'(\n|\s\s)','',bs.select('#contents_author')[0].text.strip())
    else :
        author_intro = None


    # 목차및 대표 리뷰 텍스트 파싱
    if bs.select('span.more_contents') != None and len(bs.select('span.more_contents')):
        more_conts = [];
        for tag in bs.select('span.more_contents'):
            more_conts.append(tag.text.strip())
        more_conts = " ".join(more_conts)
    else :
        more_conts = None

    #출판사 리뷰 파싱
    if( bs.find('a', attrs={'name' : 'contentsMakerReview'}) != None):
        t_cnt = 0
        for tag in bs.find('a', attrs={'name' : 'contentsMakerReview'}).next_elements:
            t_cnt+=1
            if t_cnt == 2:
                maker_review = tag.text
                break
        maker_review = re.sub(r'(\n|\s\s)','',maker_review.strip())
    else :
        maker_review = None

    bdata['book_intro'] = book_intro
    bdata['author_intro'] = author_intro
    bdata['more_contents'] = more_conts
    bdata['maker_review'] = maker_review
    bdata['url'] = url
    return bdata
    

In [22]:
uset = {}
uset['url'] = "http://www.yes24.com/24/Category/Display/001001017001"

In [19]:
links = get_links_from_url(uset)

In [30]:
for link in links:
    bdata = parse_content(link)
    print(bdata)

{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/24/Category/Display/001001014'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com//SearchCorner/Result?domain=ALL&author_yn=Y&query=Guillermo+Del+Toro'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com//SearchCorner/Result?domain=ALL&author_yn=Y&query=%b9%da%c7%fc%b1%d9'}
{'og:title': '디즈니 인 더 레인', 'og:url': 'http://www.yes24.com/campaign/01_book/yesPresent/yesP

{'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'https://ssl.yes24.com//MyPageOrderList/MyPageOrderList'}
{'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://movie.yes24.com/NMyMovie/MyMovie_Main.aspx'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/company/02_AdInfo.aspx'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/24/Category/Display/001001002'}
{'og:image': ' http://image.yes24.

{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/24/Category/Display/003001032'}
{'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://ticket.yes24.com/Pages/Genre/GenreMain.aspx?&genre=15459'}
{'og:title': '다이어터 한정판 리커버 세트', 'og:type': 'book', 'og:url': 'http://www.yes24.com/24/goods/7444744', 'og:image': 'http://image.yes24.com/goods/7444744/L', 'og:site_name': 'YES24', 'og:description': '“이보다\xa0재미있게\xa0다이어트를\xa0다룬\xa0책은\xa0없었다!”45만\xa0독자들의\xa0선택,\xa0다이어트\xa0만화의\xa0절대\xa0갑(甲),『다이어터』\xa01~3권을\xa0YES24\xa0단독\xa0리커버\xa0에디션으로\xa0만나다\xa02천만\xa0DAUM\xa0독자들이\xa0선택한\xa0다이어트의\xa0왕도\xa0만화《다이어터》가\xa0지난\xa02011년\xa08월,\xa0첫\xa0책으로\xa0출간된\xa0후\xa0뒤이어\xa02권과\xa03권(완결)이\xa0출간되어\xa045만\xa0독자들의\xa0...

{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/24/Category/Display/004001030'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/2.0/Category/CategorySeries.aspx?SeriesNumber=113443&CategoryNumber=001001017005001'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/24/Category/Display/006001090'}
{'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_content

{'og:title': '시원하게 즐기는 7월 할인 혜택', 'og:description': '퀴즈 풀면 1천원 상품권 + 카드/결제 할인까지!', 'og:url': 'http://www.yes24.com/eWorld/EventWorld/Event?eventno=157265', 'og:image': 'http://image.yes24.com/images/13_EventWorld/157265_02(1).jpg', 'og:site_name': 'YES24', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/eWorld/EventWorld/Event?eventno=157265'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/24/Category/Display/017001048'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/2.0/C

...

{'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.co.id'}
{'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://edu.yes24.com/mypage/mypage_main.aspx'}
{'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/Templates/FTMyAccount_1YESMoney.aspx'}
{'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://hansaedreams.co.kr'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url

{'og:url': 'http://www.yes24.com/campaign/00_corp/superPrice/superPrice.aspx', 'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'og:title': 'YES24 | 슈퍼특가', 'og:description': 'YES24 슈퍼특가', 'og:site_name': 'www.yes24.com', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/campaign/00_corp/superPrice/superPrice.aspx?GoodsNo=61823433'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/24/Category/Display/002001048'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com

{'og:url': 'http://www.yes24.com/campaign/00_corp/superPrice/superPrice.aspx', 'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'og:title': 'YES24 | 슈퍼특가', 'og:description': 'YES24 슈퍼특가', 'og:site_name': 'www.yes24.com', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/campaign/00_corp/superPrice/superPrice.aspx?GoodsNo=61332859'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/24/Category/Display/006001089'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com

{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/24/Category/Display/001001001'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/24/Category/Display/001001010'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/24/Category/DisplayUsed'}
{'og:title': '페이코 포인트 3% 할인', 'og:description': '페이코 포인트 3% 할인', 'og:url': 'http://www.yes24.com/eWorld/EventWorld/Event?eventno=118017', 'og:image': 'http://image.yes24.com/i

{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/searchcorner/Search?keywordAd=&keyword=&domain=BOOK&qdomain=%B1%B9%B3%BB%B5%B5%BC%AD&query=%B1%E8%BB%C7%B8%B6%B9%CC'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com/2.0/Category/CategorySeries.aspx?SeriesNumber=380&CategoryNumber=001001017001001'}
{'og:image': ' http://image.yes24.com/sysimage/renew/gnb/logoN2.png', 'cats': '', 'authors': '', 'corp': None, 'pub_date': None, 'book_intro': None, 'author_intro': None, 'more_contents': None, 'maker_review': None, 'url': 'http://www.yes24.com//SearchCorner/Result?domain=ALL&author_yn=Y&query=%c0%af%bc%d2%b